In [1]:
## LOOKING AT THE FIRMS


## DETERMINING THEIR STRENGTH IN THEIR INDUSTRY, OVERALL, AND THE KEY COMPONENTS THAT 
##    MAKE THEM POISED FOR SOLID GROWTH



In [ ]:
##############################################################################
##  INITIAL IMPORTS & SETUP

from multiprocessing import Pool
from multiprocessing import Process, Manager
from datetime import datetime
import yfinance
import mysql.connector
import pandas as pd
import csv
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import io

yes = ["Yes","yes","ye","y","Y","1"]
no = ["No","no","na","n","N","0","-1"]

pathToCompaniesCSV = 'C:/Users/User/partytime/companiesnyse.csv'
#pathToCompaniesCSV = 'C:/Users/joeca/JNN/companiesnyse.csv'

#updateCompaniesCSV()
#tickersList = []
#with open(pathToCompaniesCSV) as csvfile:
#    df = csv.reader(csvfile)
#    for row in df:
#        tickersList.append(row[0])
#print(tickersList)
tickersList = ["FB","NFLX"]


In [2]:
#############################  NLP DRIVE  ##################################



In [3]:
###########################  BUFFET ENGINE  ################################

def getMushYFF(ticker):
    YFfinancials_url = "https://finance.yahoo.com/quote/"+ticker+"/financials?p="+ticker
    YFfinancials_data = urlopen(YFfinancials_url)
    YFfinancials_html = YFfinancials_data.read()
    YFfinancials_data.close()
    page_soup = soup(YFfinancials_html, 'html.parser')
    mush = str(page_soup.body)
    tapword = "QuoteSummaryStore"
    splitword = "incomeStatementHistory"
    tailword = "FinanceConfigStore"
    f1 = mush.index(tapword)
    f2 = mush.index(tailword)
    mush = mush[f1-1:f2]
    mush = mush.split(splitword)
    ttmMush = mush[0]
    quarterMush = mush[2]
    annualMush = mush[4]
    return ttmMush, quarterMush, annualMush

def parseTTMYFF(ticker):
    parsedData = []
    ttmMush, quarterMush, annualMush = getMushYFF(ticker)
    quarterMush = quarterMush.split('},')
    undesirables = ['"','[',']','{','}','raw']
    for row in quarterMush:
        for x in undesirables:
            row = row.replace(x,'')
        row = row.replace('::',',')
        row = row.replace(':','')
        row = row.split(',')
        category = row[0]
        try:
            value = row[1]
        except IndexError:
            value = 0
        if category == '':
            proceed = 0
        elif category == 'maxAge1':
            proceed = 0
        else:
            proceed = 1
        if proceed == 1:
            parsedData.append((category,value))
    return parsedData

def parseQuarterYFF(ticker):
    parsedData = []
    ttmMush, quarterMush, annualMush = getMushYFF(ticker)
    quarterMush = quarterMush.split('},')
    undesirables = ['"','[',']','{','}','raw']
    for row in quarterMush:
        for x in undesirables:
            row = row.replace(x,'')
        row = row.replace('::',',')
        row = row.replace(':','')
        row = row.split(',')
        category = row[0]
        try:
            value = row[1]
        except IndexError:
            value = 0
        if category == '':
            proceed = 0
        elif category == 'maxAge1':
            proceed = 0
        else:
            proceed = 1
        if proceed == 1:
            parsedData.append((category,value))
    return parsedData
    
def parseAnnualYFF(ticker):
    parsedData = []
    ttmMush, quarterMush, annualMush = getMushYFF(ticker)
    annualMush = annualMush.split('},')
    undesirables = ['"','[',']','{','}','raw']
    for row in annualMush:
        for x in undesirables:
            row = row.replace(x,'')
        row = row.replace('::',',')
        row = row.replace(':','')
        row = row.split(',')
        category = row[0]
        try:
            value = row[1]
        except IndexError:
            value = 0
        if category == '':
            proceed = 0
        elif category == 'maxAge1':
            proceed = 0
        elif category == 'quoteTypeexchangeNMS':
            break
        else:
            proceed = 1
        if proceed == 1:
            parsedData.append((category,value))
    return parsedData
    
def compatability_check_YFF(tickersList):
    control = "AAPL"
    Errors = []
    control_d1_fields = []
    control_d2_fields = []
    control_d3_fields = []
    d1 = parseQuarterYFF(control)
    d2 = parseAnnualYFF(control)
    d3 = parseTTMYFF(control)
    for row in d1:
        control_d1_fields.append(row[0])
    for row in d2:
        control_d2_fields.append(row[0])
    for row in d3:
        control_d3_fields.append(row[0])
    for ticker in tickersList:
        d1 = parseQuarterYFF(ticker)
        d2 = parseAnnualYFF(ticker)
        d3 = parseTTMYFF(ticker)
        d1_fields = []
        d2_fields = []
        d3_fields = []
        for row in d1:
            d1_fields.append(row[0])
        for row in d2:
            d2_fields.append(row[0])
        for row in d3:
            d3_fields.append(row[0])
        if d1_fields not in control_d1_fields:
            Errors.append("TTM mismatch, missing from control.")
        elif control_d1_fields not in d1_fields:
            Errors.append("TTM mismatch, missing from test.")
        if d2_fields not in control_d2_fields:
            Errors.append("Quarterly mismatch, missing from control.")
        elif control_d2_fields not in d2_fields:
            Errors.append("Quarterly mismatch, missing from test.")
        if d3_fields not in control_d3_fields:
            Errors.append("Annual mismatch, missing from control.")
        elif control_d3_fields not in d3_fields:
            Errors.append("Annual mismatch, missing from test.")
    print(Errors)

In [4]:
############################  BUILD PILOT  ##################################




#############################################################################
##  DATA POPULATION FUNCTIONS DICTIONARY

popFunctions = {}

#########################################
# BUFFET DICT TTM

def fieldsbuffetdictTTM():
    tableName = 'buffetdictTTM'
    fieldNames = ["category"]
    fields = []
    placer = []
    i = 0
    for x in len(FieldNames):
        fieldName = fieldNames[i].upper()
        fields.append(tuple((str(fieldName),"VARCHAR(255)")))
        placer.append("%s")
        i += 1
    return tableName, fieldNames, fields, placer
    
def clearbuffetdictTTM():
    tableName, fieldNames, fields, placer = fieldsbuffetdictTTM()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = 'DROP TABLE ' + tableName
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    command = 'CREATE TABLE '+ tableName +' ('
    for field in fields:
        command += field[0] + " " + field[1] + ", "
    command += ")"
    command = command.replace(', )', ')')
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    
def insertFormatbuffetdictTTM():
    tableName, fieldNames, fields, placer = fieldsbuffetdictTTM()
    nameTuple = "("
    placeTuple = "("
    i = 0
    for x in fieldNames:
        nameTuple += str(x).upper() + ", "
        placeTuple += placer[i] + ", "
        i += 1
    nameTuple += ")"
    placeTuple += ")"
    nameTuple = nameTuple.replace(', )', ')')
    placeTuple = placeTuple.replace(', )', ')')
    return tableName, placeTuple

def insertManybuffetdictTTM(g):
    tableName, placeTuple = insertFormatbuffetdictTTM()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = "INSERT INTO "+ tableName +" VALUES " + placeTuple
    command = command.replace(', )', ')')
    c.executemany(command,g)
    conn.commit()

def POPbuffetdictTTM(tickersList):
    f1 = []
    for ticker in tickersList:
        d1 = parseTTMYFF(ticker)
        for row in d1:
            if row[0] not in f1:
                f1.append(row[0])
    insertManybuffetdictTTM(f1)

popFunctions['buffetdictTTM'] = POPbuffetdictTTM

#########################################
# BUFFET DICT Q

def fieldsbuffetdictQ():
    tableName = 'buffetdictQ'
    fieldNames = ["category"]
    fields = []
    placer = []
    i = 0
    for x in len(FieldNames):
        fieldName = fieldNames[i].upper()
        fields.append(tuple((str(fieldName),"VARCHAR(255)")))
        placer.append("%s")
        i += 1
    return tableName, fieldNames, fields, placer
    
def clearbuffetdictQ():
    tableName, fieldNames, fields, placer = fieldsbuffetdictQ()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = 'DROP TABLE ' + tableName
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    command = 'CREATE TABLE '+ tableName +' ('
    for field in fields:
        command += field[0] + " " + field[1] + ", "
    command += ")"
    command = command.replace(', )', ')')
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    
def insertFormatbuffetdictQ():
    tableName, fieldNames, fields, placer = fieldsbuffetdictQ()
    nameTuple = "("
    placeTuple = "("
    i = 0
    for x in fieldNames:
        nameTuple += str(x).upper() + ", "
        placeTuple += placer[i] + ", "
        i += 1
    nameTuple += ")"
    placeTuple += ")"
    nameTuple = nameTuple.replace(', )', ')')
    placeTuple = placeTuple.replace(', )', ')')
    return tableName, placeTuple

def insertManybuffetdictQ(g):
    tableName, placeTuple = insertFormatbuffetdictQ()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = "INSERT INTO "+ tableName +" VALUES " + placeTuple
    command = command.replace(', )', ')')
    c.executemany(command,g)
    conn.commit()

def POPbuffetdictQ(tickersList):
    f2 = []
    for ticker in tickersList:
        d2 = parseQuarterYFF(ticker)
        for row in d2:
            if row[0] not in f2:
                f2.append(row[0])
    insertManybuffetdictQ(f2)

popFunctions['buffetdictQ'] = POPbuffetdictQ

#########################################
# BUFFET DICT Y

def fieldsbuffetdictY():
    tableName = 'buffetdictY'
    fieldNames = ["category"]
    fields = []
    placer = []
    i = 0
    for x in len(FieldNames):
        fieldName = fieldNames[i].upper()
        fields.append(tuple((str(fieldName),"VARCHAR(255)")))
        placer.append("%s")
        i += 1
    return tableName, fieldNames, fields, placer
    
def clearbuffetdictY():
    tableName, fieldNames, fields, placer = fieldsbuffetdictY()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = 'DROP TABLE ' + tableName
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    command = 'CREATE TABLE '+ tableName +' ('
    for field in fields:
        command += field[0] + " " + field[1] + ", "
    command += ")"
    command = command.replace(', )', ')')
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    
def insertFormatbuffetdictY():
    tableName, fieldNames, fields, placer = fieldsbuffetdictY()
    nameTuple = "("
    placeTuple = "("
    i = 0
    for x in fieldNames:
        nameTuple += str(x).upper() + ", "
        placeTuple += placer[i] + ", "
        i += 1
    nameTuple += ")"
    placeTuple += ")"
    nameTuple = nameTuple.replace(', )', ')')
    placeTuple = placeTuple.replace(', )', ')')
    return tableName, placeTuple

def insertManybuffetdictY(g):
    tableName, placeTuple = insertFormatbuffetdictY()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = "INSERT INTO "+ tableName +" VALUES " + placeTuple
    command = command.replace(', )', ')')
    c.executemany(command,g)
    conn.commit()

def POPbuffetdictY(tickersList):
    f3 = []
    for ticker in tickersList:
        d3 = parseAnnualYFF(ticker)
        for row in d3:
            if row[0] not in f3:
                f3.append(row[0])
    insertManybuffetdictY(f3)

popFunctions['buffetdictY'] = POPbuffetdictY

#########################################
# BUFFET 1

def scrapebuffet1(ticker):
    try:
        symbol = yfinance.Ticker(ticker).info['symbol']
    except:
        symbol = 0
    try:
        shortName = yfinance.Ticker(ticker).info['shortName']
    except:
        shortName = 0
    try:
        country = yfinance.Ticker(ticker).info['country']
    except:
        country = 0
    try:
        state = yfinance.Ticker(ticker).info['state']
    except:
        state = 0
    try:
        industry = yfinance.Ticker(ticker).info['industry']
    except:
        industry = 0
    try:
        sector = yfinance.Ticker(ticker).info['sector']
    except:
        sector = 0
    try:
        employees = yfinance.Ticker(ticker).info['fullTimeEmployees']
    except:
        employees = 0
    try:
        volume = yfinance.Ticker(ticker).info['averageDailyVolume10Day']
    except:
        volume = 0
    try:
        sharesOutstanding = yfinance.Ticker(ticker).info['sharesOutstanding']
    except:
        sharesOutstanding = 0
    try:
        marketCap = yfinance.Ticker(ticker).info['marketCap']
    except:
        marketCap = 0
    try:
        heldPercentInstitutions = yfinance.Ticker(ticker).info['heldPercentInstitutions']
    except:
        heldPercentInstitutions = 0
    try:
        heldPercentInsiders = yfinance.Ticker(ticker).info['heldPercentInsiders']
    except:
        heldPercentInsiders = 0
    try:
        enterpriseValue = yfinance.Ticker(ticker).info['enterpriseValue']
    except:
        enterpriseValue = 0
    try:
        enterpriseToRevenue = yfinance.Ticker(ticker).info['enterpriseToRevenue']
    except:
        enterpriseToRevenue = 0
    try:
        enterpriseToEbitda = yfinance.Ticker(ticker).info['enterpriseToEbitda']
    except:
        enterpriseToEbitda = 0
    try:
        forwardEps = yfinance.Ticker(ticker).info['forwardEps']
    except:
        forwardEps = 0
    try:
        trailingEps = yfinance.Ticker(ticker).info['trailingEps']
    except:
        trailingEps = 0
    try:
        bookValue = yfinance.Ticker(ticker).info['bookValue']
    except:
        bookValue = 0
    try:
        priceToBook = yfinance.Ticker(ticker).info['priceToBook']
    except:
        priceToBook = 0
    try:
        earningsQuarterlyGrowth = yfinance.Ticker(ticker).info['earningsQuarterlyGrowth']
    except:
        earningsQuarterlyGrowth = 0
    return symbol, shortName, country, state, industry, sector, employees, volume, sharesOutstanding, marketCap, heldPercentInstitutions, heldPercentInsiders, enterpriseValue, enterpriseToRevenue, enterpriseToEbitda, forwardEps, trailingEps, bookValue, priceToBook, earningsQuarterlyGrowth

def fieldsbuffet1():
    tableName = 'buffet1'
    fieldNames = ["symbol", "shortName", "country",
                  "state", "industry", "sector", "fullTimeEmployees",
                  "averageDailyVolume10Day", "sharesOutstanding",
                  "marketCap", "heldPercentInstitutions", "heldPercentInsiders",
                  "enterpriseValue", "enterpriseToRevenue", "enterpriseToEbitda",
                  "forwardEps", "trailingEps", "bookValue", "priceToBook",
                  "earningsQuarterlyGrowth"]
    a = scrapebuffet1("MSFT")
    fields = []
    placer = []
    i = 0
    for x in a:
        fieldName = fieldNames[i].upper()
        fields.append(tuple((str(fieldName),"VARCHAR(255)")))
        placer.append("%s")
        i += 1
    return tableName, fieldNames, fields, placer
    
def clearbuffet1():
    tableName, fieldNames, fields, placer = fieldsbuffet1()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = 'DROP TABLE ' + tableName
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    command = 'CREATE TABLE '+ tableName +' ('
    for field in fields:
        command += field[0] + " " + field[1] + ", "
    command += ")"
    command = command.replace(', )', ')')
    try:
        c.execute(command)
        conn.commit()
    except:
        pass
    
def insertFormatbuffet1():
    tableName, fieldNames, fields, placer = fieldsbuffet1()
    nameTuple = "("
    placeTuple = "("
    i = 0
    for x in fieldNames:
        nameTuple += str(x).upper() + ", "
        placeTuple += placer[i] + ", "
        i += 1
    nameTuple += ")"
    placeTuple += ")"
    nameTuple = nameTuple.replace(', )', ')')
    placeTuple = placeTuple.replace(', )', ')')
    return tableName, placeTuple

def insertManybuffet1(g):
    tableName, placeTuple = insertFormatbuffet1()
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = "INSERT INTO "+ tableName +" VALUES " + placeTuple
    command = command.replace(', )', ')')
    c.executemany(command,g)
    conn.commit()

def POPbuffet1(tickersList):
    clearbuffet1()
    g = []
    for ticker in tickersList:
        startTime = datetime.now()
        h = scrapebuffet1(ticker)
        g.append(h)
        endTime = datetime.now()
        elapsed = endTime - startTime
        print("Elapsed Time for Scrape:  " + str(elapsed))
    insertManybuffet1(g)
    
popFunctions['buffet1'] = POPbuffet1

#########################################
# BUFFET 2

def POPbuffet2(tickersList):
    print("Fire Hose Protocol: Test Successful.")
popFunctions['buffet2'] = POPbuffet2



##############################################################################


##############################################################################
##  DATABASE CONSTRUCTION FUNCTIONS

def viewAll(tableName):
    conn = mysql.connector.connect(user='jupiter', password='###########',
                              host='localhost',
                              database='jnn')
    c = conn.cursor()
    command = "SELECT * FROM " + tableName
    c.execute(command)
    allData = c.fetchall()
    conn.commit()
    print(allData)

        
##############################################################################


##############################################################################
##  DATABASE UPDATING FUNCTIONS

def updateCompaniesCSV():
    import urllib
    urllib.request.urlretrieve('https://dumbstockapi.com/stock?format=csv&countries=US', pathToCompaniesCSV)
    
def populateTable(tableName):
    proceed = input("Commence populating table: "+tableName+"?")
    while True:
        if proceed in yes:
            pop = popFunctions[tableName]
            pop()
            print("Data Population Successful.")
            break
        elif proceed not in no:
            proceed = input("Invalid answer. Commence populating table: "+tableName+"?")
        else:
            print("Aborting Data Population...")
            break
    
def FireHoseProtocol(passcode):
    if str(passcode) == str(915):
        proceed = input("Are you sure you want to activate Fire Hose Protocol?")
        FHstartTime = datetime.now()
        while True:
            if proceed in yes:
                print("Fire Hose Protocol: Activated.")
                for pop in popFunctions.values():
                    pop(tickersList)
                print("Fire Hose Protocol: Data Population Successful.")
                break
            elif proceed not in no:
                proceed = input("Invalid answer. Are you sure you want to activate Fire Hose Protocol?")
            else:
                print("Aborting Fire Hose Protocol...")
                break
        FHendTime = datetime.now()
        FHelapsed = FHendTime - FHstartTime
        print("Elapsed Time for Protocol :  " + str(FHelapsed))
    else:
        print("Access Denied. Incorrect access code.")    

##############################################################################

##############################################################################
##

In [5]:
FireHoseProtocol(915)

Are you sure you want to activate Fire Hose Protocol?y
Fire Hose Protocol: Activated.
Elapsed Time for Scrape:  0:00:59.283932
Elapsed Time for Scrape:  0:00:56.241654
Elapsed Time for Scrape:  0:00:57.449227
Elapsed Time for Scrape:  0:00:54.017682
Elapsed Time for Scrape:  0:00:58.297878
Fire Hose Protocol: Test Successful.
Fire Hose Protocol: Data Population Successful.
Elapsed Time for Protocol :  0:06:43.298582


In [6]:
viewAll("buffet1")

[('FB', 'Facebook, Inc.', 'United States', 'CA', 'Internet Content & Information', 'Communication Services', '52534', '19678350', '2404280064', '761632063488', '0.79412', '0.006640000000000001', '719123972096', '9.568', '21.099', '10.12', '8.178', '38.753', '6.8995953', '0.979'), ('NFLX', 'Netflix, Inc.', 'United States', 'CA', 'Entertainment', 'Communication Services', '8600', '4116762', '441015008', '216917639168', '0.8348700000000001', '0.016220000000000002', '230136496128', '10.17', '59.549', '8.8', '5.922', '21.166', '23.238684', '1.661'), ('TSLA', 'Tesla, Inc.', 'United States', 'CA', 'Auto Manufacturers', 'Consumer Cyclical', '48016', '16622000', '186362000', '389263622144', '0.5792700000000001', '0.2051', '381408509952', '14.836', '110.714', '15.55', '1.932', '52.984', '39.43568', None), ('GRUB', 'GrubHub Inc.', 'United States', 'IL', 'Internet Content & Information', 'Communication Services', '2714', '1243037', '92277696', '7015873024', '1.1182400000000001', '0.0038', '7196009